<a href="https://www.kaggle.com/code/bradyskate/bunkr-downloader-downscaler?scriptVersionId=186023056" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#(CELL 1)
# @title INSTALL REQUIRED LIBRARIES AND FFMPEG WITH GPU SUPPORT
!pip install tqdm
!pip install ffmpeg-python
!apt update -qq && apt install -y ffmpeg
!pip install --upgrade --force-reinstall --no-deps 'git+https://github.com/kkroening/ffmpeg-python.git'
# Ensure CUDA and NVIDIA drivers are installed
!apt-get install -qq cuda

83 packages can be upgraded. Run 'apt list --upgradable' to see them.



ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.
  Cloning https://github.com/kkroening/ffmpeg-python.git to /tmp/pip-req-build-tzy8b3an
  Running command git clone --filter=blob:none --quiet https://github.com/kkroening/ffmpeg-python.git /tmp/pip-req-build-tzy8b3an
  Resolved https://github.com/kkroening/ffmpeg-python.git to commit df129c7ba30aaa9ffffb81a48f53aa7253b0b4e6
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for ffmpeg-python: filename=ffmpeg_python-0.2.0-py3-none-any.whl size=25327 sha256=4354d3ce51734a150992d2ad19d51601ee5dcf727d2303f2a54334a55217bbc6
  Stored in directory: /tmp/pip-ephem-wheel-cache-2w_9x123/wheels/0c/93/fb/4f66aa585b6b9428e1eb3d2e8a02c51cd81b1efe6e37e49e11
Successfully built ffmpe

In [2]:
# @title SPECIFY VIDEO URL, SAVE PATH, DOWNLOAD AND DOWNSCALE FUNCTION
import requests
from tqdm import tqdm
import ffmpeg
import os

# Define the video URL
video_url = 'https://pizza.bunkr.ru/Tatiana_Fanbus-l8b2FGCK.mp4'  # @param {type:"string"} 
# Replace with your video URL
# Define the save path
save_option = "default"  # @param ["default", "gdrive"] {type:"string"}

# Path based on the environment
if save_option == "gdrive":
    save_path = '/content/drive/MyDrive/video.mp4'  # Google Drive path
else:
    # Uncomment the appropriate path based on the environment
    save_path = '/kaggle/working/video.mp4'  # Kaggle default path
    # save_path = '/content/video.mp4'  # Colab default path
def download_video(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte
    t = tqdm(total=total_size, unit='iB', unit_scale=True)
    with open(save_path, 'wb') as file:
        for data in response.iter_content(block_size):
            t.update(len(data))
            file.write(data)
    t.close()
    if total_size != 0 and t.n != total_size:
        print("ERROR, something went wrong")
    else:
        print(f"Video downloaded to {save_path}")
download_video(video_url, save_path)

# Define the desired resolution
desired_resolution = "480p"  # @param ["480p", "360p"] {type:"string"}
def downscale_video(input_path, output_path, resolution="480p"):
    try:
        # Map resolution options to scale values
        resolution_map = {
            "480p": "854x480",
            "360p": "640x360"
        }
        
        if resolution not in resolution_map:
            raise ValueError("Invalid resolution. Choose either '480p' or '360p'.")
        scale_value = resolution_map[resolution]
        
        # Probe the video to get file info
        probe = ffmpeg.probe(input_path)
        video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
        total_frames = int(video_info['nb_frames'])
        
        # Function to update the progress bar
        def progress_callback(stats):
            frame = int(stats.get('frame', 0))
            t.update(frame - t.n)
        
        # Create a progress bar
        with tqdm(total=total_frames, unit='frame') as t:
            # Use h264_nvenc codec for GPU encoding
            process = (
                ffmpeg
                .input(input_path)
                .output(output_path, vf=f"scale={scale_value}", vcodec="h264_nvenc", preset="fast")
                .global_args('-progress', 'pipe:1', '-nostats')
                .global_args('-y')  # Overwrite output files without asking
                .global_args('-loglevel', 'error')  # Suppress ffmpeg console output
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
            )
            for line in process.stdout:
                line = line.decode().strip()
                if 'frame=' in line:
                    stats = {}
                    for item in line.split(' '):
                        key, value = item.split('=')
                        stats[key] = value
                    progress_callback(stats)
        
        print(f"Downscaled video saved to {output_path}")
        
        # Print original and downscaled file sizes
        original_size = os.path.getsize(input_path) / (1024 * 1024)
        downscaled_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f"Original size: {original_size:.2f} MB")
        print(f"Downscaled size: {downscaled_size:.2f} MB")
    
    except ffmpeg.Error as e:
        # Print the full error message for debugging
        print(f"ffmpeg error: {e.stderr.decode()}")
    except ValueError as ve:
        # Handle invalid resolution error
        print(ve)
# Define the path to your output video file
output_video_path = save_path.replace('.mp4', '_downscaled.mp4')
downscale_video(save_path, output_video_path, resolution=desired_resolution)

100%|██████████| 265M/265M [00:10<00:00, 24.7MiB/s]


Video downloaded to /kaggle/working/video.mp4


100%|██████████| 14807/14807 [00:52<00:00, 282.99frame/s]

Downscaled video saved to /kaggle/working/video_downscaled.mp4
Original size: 253.10 MB
Downscaled size: 156.72 MB


In [3]:
# @title VERIFY GPU USAGE
import subprocess
def check_gpu_usage():
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    print(result.stdout.decode())
check_gpu_usage()

Sat Jun 29 06:34:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0              28W /  70W |      0MiB / 15360MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# @title STREAM DOWNLOADED VIDEO FILE
from IPython.display import HTML
# Define the path to the downloaded video file
# Ensure this path matches the path used in the download function
save_option = "default"  # @param ["default", "gdrive"] {type:"string"}
# Path based on the environment
if save_option == "gdrive":
    save_path = '/content/drive/MyDrive/video.mp4'  # Google Drive path
else:
    # Uncomment the appropriate path based on the environment
    save_path = '/kaggle/working/video.mp4'  # Kaggle default path
    # save_path = '/content/video.mp4'  # Colab default path
# Generate HTML to embed the video
html_video = f"""
<video width="640" height="480" controls>
  <source src="{save_path}" type="video/mp4">
  Your browser does not support the video tag.
</video>
"""
# Display the video player
HTML(html_video)